In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, f1_score, classification_report
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/ptb-xl-tcc')

Mounted at /content/drive


Carregando dados de treino e teste

In [ ]:
X_train = np.load('/content/drive/MyDrive/ptb-xl-tcc/outfile_X_train.npy', allow_pickle=True)
X_test = np.load('/content/drive/MyDrive/ptb-xl-tcc/outfile_X_test.npy', allow_pickle=True)
y_train = np.load('/content/drive/MyDrive/ptb-xl-tcc/outfile_y_train.npy', allow_pickle=True)
y_test = np.load('/content/drive/MyDrive/ptb-xl-tcc/outfile_y_test.npy', allow_pickle=True)

Função que remove elementos vazios da lista

In [ ]:
def remover(X,y):
     aux = []
     ret = 0
     x_aux = []
     y_aux = []
     for i in range(len(y)):
         if y[i] != []:
           y_aux.append(y[i])
           x_aux.append(X[i])
     return np.array(x_aux), np.array(y_aux)

In [ ]:
X_train,y_train = remover(X_train,y_train)
X_test,y_test = remover(X_test,y_test)

<ipython-input-4-c44d1cdeb708>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(x_aux), np.array(y_aux)


In [ ]:
print("X_train:" + str(X_train.shape) + "; y_train:" + str(y_train.shape) + "; X_test:" + str(X_test.shape) + "; y_test:" + str(y_test.shape))

X_train:(19237, 1000, 12); y_train:(19237,); X_test:(2159, 1000, 12); y_test:(2159,)


Função que realiza codificação multi-hot

In [ ]:
def multi_hot_encoding(y):
    #Obter dados categóricos
    y_encoded = np.array(y)

    # Criar uma instância do codificador multi-hot
    encoder = MultiLabelBinarizer()

    # Aplicar a codificação multi-hot aos dados
    y_encoded_ = encoder.fit_transform(y_encoded)

    return y_encoded_

In [ ]:
y_train = multi_hot_encoding(y_train)
y_test = multi_hot_encoding(y_test)

In [ ]:
y_train

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0]])

In [ ]:
y_train.shape

(19237, 5)

In [ ]:
y_test

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

In [ ]:
y_test.shape

(2159, 5)

Arquitetura Transformer

In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, input_dim, n_heads, ff_dim, dropout=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.attention = layers.MultiHeadAttention(
            num_heads=n_heads, key_dim=input_dim[-1] // n_heads
        )
        self.ff = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(dropout),
        ])

    def build(self, input_shape):
        #Adiciona a camada densa residual
        self.dense_residual = layers.Dense(input_shape[-1])

    def call(self, inputs):
        attention_output = self.attention(inputs, inputs)
        x = tf.keras.layers.Add()([attention_output, inputs])  #Conexão residual

        #Ajuste das dimensões usando Dense
        residual = self.dense_residual(x)
        x = tf.keras.layers.Add()([x, residual])  #Conexão residual

        x = self.ff(x)
        return x


# Definindo o modelo Transformer
class Transformer(keras.Model):
    def __init__(self, n_blocks, n_heads, ff_dim, n_classes, input_shape, rate=0.1):
        super(Transformer, self).__init__()
        self.encoder_layers = [TransformerBlock(input_shape, n_heads, ff_dim, rate) for _ in range(n_blocks)]
        self.flatten = layers.Flatten()
        self.f_connected = layers.Dense(n_classes, activation="sigmoid")

    def call(self, inputs):
        x = inputs
        for layer in self.encoder_layers:
            x = layer(x)
        x = self.flatten(x)
        x = self.f_connected(x)
        return x

In [ ]:
input_dim = X_train.shape[1:]
n_blocks = 6
n_heads = 4
ff_dim = 512
n_classes = 5

#Criar instância do modelo Transformer
model = Transformer(n_blocks=n_blocks, n_heads=n_heads, ff_dim=ff_dim, n_classes=n_classes, input_shape=input_dim, rate=0.1)

#Criar um tensor de entrada baseado nas dimensões desejadas
input_dim_ = tf.keras.Input(shape=input_dim)
model(input_dim_)  # Chamando o modelo com o tensor de entrada

#Compilar o modelo depois de chamar o modelo com o tensor de entrada
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss="binary_crossentropy",
              metrics=["accuracy",
                        tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall(),
              ])
model.summary()


Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer_block (Transfo  multiple                  9484      
 rmerBlock)                                                      
                                                                 
 transformer_block_1 (Trans  multiple                  552484    
 formerBlock)                                                    
                                                                 
 transformer_block_2 (Trans  multiple                  552484    
 formerBlock)                                                    
                                                                 
 transformer_block_3 (Trans  multiple                  552484    
 formerBlock)                                                    
                                                                 
 transformer_block_4 (Trans  multiple                  

Realizando treinamento do modelo

In [ ]:
epochs = 30
batch_size = 32
early_stopping = EarlyStopping(patience=10, min_delta=1e-3, monitor="val_accuracy", restore_best_weights=True)


history = model.fit(X_train,
                    y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[early_stopping]
)

Epoch 1/30
542/542 [==============================] - 361s 629ms/step - loss: 36.1300 - accuracy: 0.3012 - precision: 0.3323 - recall: 0.3174 - val_loss: 0.7343 - val_accuracy: 0.3373 - val_precision: 0.2739 - val_recall: 0.0168
Epoch 2/30
542/542 [==============================] - 340s 628ms/step - loss: 0.6659 - accuracy: 0.3787 - precision: 0.4109 - recall: 0.1811 - val_loss: 0.6045 - val_accuracy: 0.3347 - val_precision: 0.3544 - val_recall: 0.2543
Epoch 3/30
542/542 [==============================] - 333s 614ms/step - loss: 0.6030 - accuracy: 0.3989 - precision: 0.4345 - recall: 0.1816 - val_loss: 0.6642 - val_accuracy: 0.3363 - val_precision: 0.3564 - val_recall: 0.2680
Epoch 4/30
542/542 [==============================] - 340s 628ms/step - loss: 0.6259 - accuracy: 0.3821 - precision: 0.4295 - recall: 0.1987 - val_loss: 0.6704 - val_accuracy: 0.2531 - val_precision: 0.2929 - val_recall: 0.1020
Epoch 5/30
542/542 [==============================] - 341s 629ms/step - loss: 0.6380 - 

Salvar o melhor modelo com base em val_accuracy

In [ ]:
model.save("transformer_3_", save_format="tf")

Análise do modelo

In [ ]:
history.history

{'loss': [36.1299934387207,
  0.6659169793128967,
  0.6030179858207703,
  0.6259499192237854,
  0.6380007266998291,
  0.636443018913269,
  0.5725260972976685,
  0.7819646000862122,
  0.6491446495056152,
  1.48589026927948,
  1.2972437143325806],
 'accuracy': [0.30121874809265137,
  0.37873274087905884,
  0.39889100193977356,
  0.382140576839447,
  0.3482931852340698,
  0.3562640845775604,
  0.38294923305511475,
  0.34390342235565186,
  0.3459250330924988,
  0.30041009187698364,
  0.34424999356269836],
 'precision': [0.33229073882102966,
  0.4109076261520386,
  0.43454602360725403,
  0.4294803738594055,
  0.40566039085388184,
  0.40924885869026184,
  0.4572760760784149,
  0.3831206262111664,
  0.40135180950164795,
  0.33190539479255676,
  0.3759653568267822],
 'recall': [0.3174419105052948,
  0.18110868334770203,
  0.18164384365081787,
  0.19867992401123047,
  0.20519110560417175,
  0.20483432710170746,
  0.18806582689285278,
  0.20751014351844788,
  0.20920483767986298,
  0.25224101543

In [ ]:
evaluation_results = model.evaluate(X_test, y_test)

68/68 [==============================] - 15s 215ms/step - loss: 0.7139 - accuracy: 0.4145 - precision: 0.2933 - recall: 0.0297


In [ ]:
y_pred = model.predict(X_test, verbose=False)

In [ ]:
y_pred

array([[0.11489141, 0.00108702, 0.17133239, 0.28700426, 0.21294896],
       [0.10129156, 0.00161635, 0.16632022, 0.32325423, 0.18510239],
       [0.10137324, 0.00142326, 0.16277047, 0.31938222, 0.18809947],
       ...,
       [0.09730214, 0.00150658, 0.16006103, 0.3288494 , 0.18358906],
       [0.09857244, 0.00145061, 0.16070573, 0.3247684 , 0.18504421],
       [0.10305209, 0.00155862, 0.16719405, 0.31854096, 0.18844104]],
      dtype=float32)

In [ ]:
y_test

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

In [ ]:
#define limiar(threshold) para converter as previsões contínuas em rótulos binários e poder usar o classification_report
threshold = 0.3

#aplicando o limiar às previsões
y_pred_ = (y_pred > threshold).astype(int)

print(classification_report(y_test, y_pred_))

              precision    recall  f1-score   support

           0       0.31      0.04      0.08       497
           1       0.11      0.01      0.02       262
           2       0.28      0.05      0.08       551
           3       0.45      0.85      0.58       963
           4       0.18      0.02      0.04       521

   micro avg       0.42      0.32      0.36      2794
   macro avg       0.27      0.20      0.16      2794
weighted avg       0.31      0.32      0.24      2794
 samples avg       0.39      0.39      0.38      2794



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.35062528948587307


In [ ]:
#define limiar(threshold) para converter as previsões contínuas em rótulos binários e poder usar o classification_report
threshold = 0.5

#aplicando o limiar às previsões
y_pred_ = (y_pred > threshold).astype(int)

print(classification_report(y_test, y_pred_))

              precision    recall  f1-score   support

           0       0.28      0.03      0.06       497
           1       0.06      0.00      0.01       262
           2       0.31      0.03      0.06       551
           3       0.37      0.04      0.07       963
           4       0.19      0.01      0.03       521

   micro avg       0.29      0.03      0.05      2794
   macro avg       0.24      0.03      0.05      2794
weighted avg       0.28      0.03      0.05      2794
 samples avg       0.02      0.03      0.03      2794



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.014358499305233904


In [ ]:
#define limiar(threshold) para converter as previsões contínuas em rótulos binários e poder usar o classification_report
threshold = 0.75

#aplicando o limiar às previsões
y_pred_ = (y_pred > threshold).astype(int)

print(classification_report(y_test, y_pred_))

              precision    recall  f1-score   support

           0       0.34      0.03      0.06       497
           1       0.07      0.00      0.01       262
           2       0.27      0.02      0.04       551
           3       0.40      0.03      0.05       963
           4       0.12      0.01      0.01       521

   micro avg       0.30      0.02      0.04      2794
   macro avg       0.24      0.02      0.03      2794
weighted avg       0.28      0.02      0.04      2794
 samples avg       0.02      0.02      0.02      2794



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.012505789717461788


In [ ]:
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.28      0.03      0.06       497
           1       0.00      0.00      0.00       187
           2       0.18      0.02      0.04       317
           3       0.43      0.95      0.59       916
           4       0.12      0.02      0.04       242

    accuracy                           0.41      2159
   macro avg       0.20      0.20      0.14      2159
weighted avg       0.29      0.41      0.27      2159



In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.012505789717461788
